In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [2]:
!pip install --upgrade transformers datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/7c/55/b3432f43d6d7fee999bb23a547820d74c48ec540f5f7842e41aa5d8d5f3a/datasets-2.14.6-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [3]:
#load the dataset
from datasets import load_dataset , DatasetDict
data = load_dataset('csv', data_files='/kaggle/input/contradictory-my-dear-watson/train.csv', split='train')
data = data.train_test_split(0.2)

data = DatasetDict({
    'train': data['train'],
    'val': data['test'],
    'test':load_dataset('csv', data_files='/kaggle/input/contradictory-my-dear-watson/test.csv', split='train')
})

data

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label'],
        num_rows: 9696
    })
    val: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label'],
        num_rows: 2424
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language'],
        num_rows: 5195
    })
})

In [4]:
df = data['train'].to_pandas()
df

,id,premise,hypothesis,lang_abv,language,label
0,c2e833ed12,She was alone at last with the president!,"At last, she has been alone with the president!",en,English,0
1,408f0410f5,"But employers are still driving, and that's al...",Employers have continued to operate motor vehi...,en,English,0
2,a934f4bc79,His off-the-cuff style seems amateurish next t...,He didn't look like an amateur,en,English,2
3,7b49d5282e,This one ended up being surprisingly easy!,This was a really hard one.,en,English,2
4,7125b36327,the the Iranian borders are still open uh from...,Iran let their borders open due to a crisis.,en,English,1
...,...,...,...,...,...,...
9691,5d2e32ee8c,Since his death it has been transformed into t...,The Marley family burned the museum down after...,en,English,2
9692,3dd07b1b9e,6 Суд умеренно пользовался данным полномочием ...,Суд время от времени пользовался своими полном...,ru,Russian,0
9693,97ae730f80,"First, we can acknowledge, and maybe even do s...",We can acknowledge there is no fallout from gl...,en,English,2
9694,e385086002,"Otros respondieron, pero Keyes la lio.",Keyes dio más información que el resto.,es,Spanish,1


In [5]:
df[['premise', 'hypothesis', 'lang_abv', 'language']].describe()


,premise,hypothesis,lang_abv,language
count,9696,9696,9696,9696
unique,7065,9696,15,15
top,Trays can be found in all sizes and those with...,"At last, she has been alone with the president!",en,English
freq,3,1,5502,5502


In [6]:
df[['lang_abv', 'language']].value_counts()


lang_abv  language  
en        English       5502
zh        Chinese        338
ur        Urdu           318
ar        Arabic         310
fr        French         305
sw        Swahili        304
vi        Vietnamese     302
hi        Hindi          297
ru        Russian        297
th        Thai           297
es        Spanish        294
tr        Turkish        294
el        Greek          291
bg        Bulgarian      278
de        German         269
Name: count, dtype: int64

In [7]:
df.isna().sum()


id            0
premise       0
hypothesis    0
lang_abv      0
language      0
label         0
dtype: int64

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

checkpoint = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'
# checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, truncation=True)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [9]:
data


DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label'],
        num_rows: 9696
    })
    val: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label'],
        num_rows: 2424
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language'],
        num_rows: 5195
    })
})

In [10]:
def preprocess_data(batch):
    batch['text'] = [x1 + f' {tokenizer.sep_token} ' + x2 for x1,x2 in zip(batch['premise'],batch['hypothesis'])]
    return tokenizer(batch['text'])

data = data.map(preprocess_data, batched=True)
data


Map:   0%|          | 0/9696 [00:00<?, ? examples/s]

Map:   0%|          | 0/2424 [00:00<?, ? examples/s]

Map:   0%|          | 0/5195 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 9696
    })
    val: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 2424
    })
    test: Dataset({
        features: ['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'text', 'input_ids', 'attention_mask'],
        num_rows: 5195
    })
})

In [11]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_metric


def compute_metrics(eval_pred):
    metric = load_metric('accuracy')
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=labels)


train_args = TrainingArguments(
    output_dir='results',
    logging_dir='logs',
    report_to=[],
    
    evaluation_strategy='steps',
    eval_steps=200,
    logging_steps=200,
    save_steps=200,
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.005,
    num_train_epochs=5,
    fp16=True,
    
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    save_total_limit=3,
)

train_args

TrainingArguments(
_n_gpu=2,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=200,
evaluation_strategy=steps,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=True,
group_by_length=False,
half_precision_backend=au

In [12]:
trainer = Trainer(
    model=model,
    args=train_args,
    data_collator=data_collator,
    train_dataset=data['train'],
    eval_dataset=data['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(3)]
)

trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
200,0.335200,0.260494,0.902640
400,0.253500,0.311661,0.894802
600,0.213000,0.291684,0.898515
800,0.133500,0.404667,0.893977


/tmp/ipykernel_27/3248437400.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=800, training_loss=0.23381588935852052, metrics={'train_runtime': 554.9431, 'train_samples_per_second': 87.36, 'train_steps_per_second': 2.73, 'total_flos': 1250134070568768.0, 'train_loss': 0.23381588935852052, 'epoch': 2.64})

In [13]:
preds = trainer.predict(data['test'])
preds

PredictionOutput(predictions=array([[-3.2045007, -2.5771832,  5.809753 ],
       [-3.6085591,  5.7127914, -2.8247805],
       [ 5.3068438, -1.8113503, -2.4559202],
       ...,
       [ 5.556204 , -2.5978787, -1.8031595],
       [ 3.347456 , -1.3297172, -1.2461293],
       [-2.7143285, -1.106406 ,  3.7669308]], dtype=float32), label_ids=None, metrics={'test_runtime': 28.7007, 'test_samples_per_second': 181.006, 'test_steps_per_second': 5.679})

In [14]:
pd.DataFrame({
    'id': data['test']['id'],
    'prediction': np.argmax(preds.predictions, axis=1)
}).to_csv('submission.csv', header=True, index=False)